In [4]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4084, done.
remote: Counting objects: 100% (4084/4084), done.
remote: Compressing objects: 100% (3076/3076), done.
remote: Total 4084 (delta 1185), reused 1984 (delta 948), pack-reused 0
Receiving objects: 100% (4084/4084), 44.62 MiB | 3.78 MiB/s, done.
Resolving deltas: 100% (1185/1185), done.


In [5]:
cd models/research

/home/samruddhak/models/research


In [6]:
!protoc object_detection/protos/*.proto --python_out=.

In [8]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [10]:
!pip install tf_slim 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 2.0 MB/s eta 0:00:00m eta 0:00:010:00:01m


In [11]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [13]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [14]:
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [15]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('object_detection/test_images/image1.jpg'),
 PosixPath('object_detection/test_images/image2.jpg'),
 PosixPath('object_detection/test_images/image3.jpg')]

In [42]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [17]:
print(detection_model.inputs)
detection_model.output_dtypes

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


{'detection_boxes': tf.float32,
 'num_detections': tf.float32,
 'detection_scores': tf.float32,
 'detection_classes': tf.float32}

In [24]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]
  output_dict = model(input_tensor)
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [39]:
def show_inference(model, image_path,class_id):
  image_np = np.array(Image.open(image_path))
  output_dict = run_inference_for_single_image(model, image_np)
  boxes = []
  classes = []
  scores = []
  for i,x in enumerate(output_dict['detection_classes']):
    if x==class_id and output_dict['detection_scores'][i] > 0.5:
      classes.append(x)
      boxes.append(output_dict['detection_boxes'][i])
      scores.append(output_dict['detection_scores'][i])
  boxes = np.array(boxes)
  classes = np.array(classes)
  scores = np.array(scores)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      classes,
      scores,
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2)
  
  # display(Image.fromarray(image_np))
  print(boxes)
  print(classes)
  print(scores)
  print(output_dict)

In [40]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path, 18)

[[0.03908402 0.01921503 0.8721036  0.3157735 ]
 [0.1095151  0.4028356  0.92464614 0.97304785]]
[18 18]
[0.94069046 0.9345026 ]
{'detection_boxes': array([[3.90840173e-02, 1.92150325e-02, 8.72103572e-01, 3.15773487e-01],
       [1.09515101e-01, 4.02835608e-01, 9.24646139e-01, 9.73047853e-01],
       [5.07123709e-01, 3.85651529e-01, 8.76479745e-01, 7.03940928e-01],
       [6.38730824e-02, 2.81166643e-01, 9.16385174e-01, 8.24097157e-01],
       [2.15276688e-01, 3.96740973e-01, 9.12904143e-01, 6.79073751e-01],
       [5.27002215e-01, 3.72713268e-01, 8.83502841e-01, 6.94296658e-01],
       [0.00000000e+00, 5.25425553e-01, 2.53364950e-01, 7.16124296e-01],
       [7.37803876e-02, 2.98963994e-01, 9.20516491e-01, 9.38026786e-01],
       [5.31044006e-01, 3.84130120e-01, 9.15436625e-01, 8.53949070e-01],
       [1.45327568e-01, 3.87902439e-01, 9.06316280e-01, 9.78849113e-01],
       [2.08420515e-01, 2.78770208e-01, 8.26954961e-01, 6.09079719e-01],
       [3.48854154e-01, 3.05225760e-01, 8.99384618